This Notebook is used to analyze and generate a rendering for the first 4 scores of Ludus Musicalis by Roman Haubenstock-Ramati.

The scores physical size is ~20x20 cm (?) (square aspect ratio).

They can be read from 4 different directions, and can be performed simultaneously or in sequence.

The companion document explain the meaning of the symbols and how the pieces should be performed.

In [ ]:
# First import the MTM package and check its version: pip install multi-template-matching
# Should be 2.0.1 as 2.0.0 had a bug
# It fails if you already have OpenCV (remove it first)
import MTM
from MTM import drawBoxesOnRGB, matchTemplates
print("MTM version: ", MTM.__version__)

# But we need OpenCV anyway for image processing so we must reinstall it after: pip install opencv-python
import cv2
from scipy.ndimage import rotate
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski

# For the gamma curve
# pip install splines
import splines

# PyWebView is used for dialog boxes: pip install pywebview
import webview

# mido is used to generate midi files and messages
# On Win: pip install mido[ports-rtmidi]
# On Mac: pip install python-rtmidi 
#       pip install mido
from mido import MetaMessage, Message, MidiFile, MidiTrack

# For creating scores
import music21
# Music 21 does not recognize MuseScore 4 yet, set the correct location
# On Windows:
# music21.environment.set('musescoreDirectPNGPath', 'C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe')
# On Mac:
# TODO

# With PyWebView we prepare the file dialog to show only the supported image types.
def webview_file_dialog():
    file = None
    file_types = ('Image Files (*.bmp;*.jpg;*.gif)', 'All files (*.*)')
    def open_file_dialog(w):
        nonlocal file
        try:
            file = w.create_file_dialog(webview.OPEN_DIALOG, allow_multiple=False, file_types=file_types)[0]
        except TypeError:
            pass  # user exited file dialog without picking
        finally:
            w.destroy()
    window = webview.create_window("", hidden=True)
    webview.start(open_file_dialog, window)
    # file will either be a string or None
    return file

# Custom image scaling function
def scale_image(image, percent, maxwh):
    max_width = maxwh[1]
    max_height = maxwh[0]
    max_percent_width = max_width / image.shape[1] * 100
    max_percent_height = max_height / image.shape[0] * 100
    max_percent = 0
    if max_percent_width < max_percent_height:
        max_percent = max_percent_width
    else:
        max_percent = max_percent_height
    if percent > max_percent:
        percent = max_percent
    width = int(image.shape[1] * percent / 100)
    height = int(image.shape[0] * percent / 100)
    result = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
    return result, percent

We present the user with the choice of analyzing one of the first 4 scores, supplying another image, or acquiring an image with a webcam.

In [387]:
scoreNumber=int(input("Enter the score number that you want to analyze (1-2 Punkte, 3-4 Pizzicati, 5 Other files, 6 Camera)"))

In [388]:
#Set search files and bounding boxes
filenameSearch=''
filenameTemplates=''

match scoreNumber:
    #Punkte
    case 1:
        filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis1DPI300Gamma.jpg'
        searchBoxX=250
        searchBoxY=305
        searchBoxWidth=1990
        searchBoxHeight=2030
        ruleSet=1
    case 2:
        filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis2DPI300Gamma.jpg'
        searchBoxX=250
        searchBoxY=290
        searchBoxWidth=2000
        searchBoxHeight=2030
        ruleSet=1
    # Pizzicati
    case 3:
        filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis3DPI300Gamma.jpg'
        searchBoxX=240
        searchBoxY=280
        searchBoxWidth=2000
        searchBoxHeight=2050
        ruleSet=2
    case 4:
        filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis4DPI300Gamma.jpg'
        searchBoxX=250
        searchBoxY=280
        searchBoxWidth=2000
        searchBoxHeight=2050
        ruleSet=2
    case 5:
        filenameSearch=webview_file_dialog()
        if filenameSearch==None:
            raise SystemExit("No File Selected")
        
        ruleSet=int(input("Enter the set of rules you want to adopt (1 Punkte, 2 Pizzicati)"))
        if ruleSet==2:
            filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        elif ruleSet==1:
            filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        else :
            raise SystemExit("Invalid Input")
        # ScanNewPizzicati.jpg coordinates
        searchBoxX=350
        searchBoxY=100
        searchBoxWidth=2300
        searchBoxHeight=2290

    case 6:
        print("Opening camera")

    case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
        raise SystemExit("Invalid Input")

if scoreNumber==6:
        # If you pass an unreasonably high values for the resolution the camera will return the highest available.
        HIGH_VALUE = 10000
        WIDTH = HIGH_VALUE
        HEIGHT = HIGH_VALUE

        capture = cv2.VideoCapture(0)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        capture.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        capture.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
        width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        cv2.namedWindow("Acquire an image")
        # On Mac this seems to be needed
        cv2.startWindowThread()
        while True:
            ret, frame = capture.read()
            if not ret:
                print("Failed to grab frame")
                break
            cv2.imshow("Acquire an image", frame)

            k = cv2.waitKey(1)
            if k%256 == 27:
                # ESC pressed
                print("Escape hit, closing...")
                break
            elif k%256 == 32:
                # SPACE pressed
                print("Image acquired.")
                musicSearch=frame
    
        capture.release()
        # Workaround for Mac...
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)

        # Apply a color curve
        xvals = 0, 144, 192, 196, 255
        yvals = 0, 0, 192, 255, 255

        # Derive Catmull-Rom spline for our X,Y
        res = splines.CatmullRom(yvals, xvals)

        # Make LUT (Lookup Table) from spline
        LUT = np.uint8(res.evaluate(range(0,256)))
        # TODO fix the gamma curve
        # musicSearch = cv2.LUT(musicSearch, LUT)
        musicSearch=cv2.cvtColor(musicSearch, cv2.COLOR_RGB2BGR)
        musicVisualize=musicSearch.copy()

        ruleSet=int(input("Enter the set of rules you want to adopt (1 Punkte, 2 Pizzicati)"))
        if ruleSet==2:
            filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        elif ruleSet==1:
            filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        else :
            raise SystemExit("Invalid Input")
        # ScanNewPizzicati.jpg coordinates
        searchBoxX=350
        searchBoxY=100
        searchBoxWidth=2300
        searchBoxHeight=2290

musicTemplate = ski.io.imread(filenameTemplates)
musicVisualizeTemplate=musicTemplate.copy()
if scoreNumber!=6:
    musicSearch = ski.io.imread(filenameSearch)
    musicVisualize = musicSearch.copy()

if (scoreNumber==5 or scoreNumber==6):
    # This allows to rescale the image to fit into the screen
    cv2.namedWindow("Select Bounding Box", cv2.WINDOW_NORMAL)
    cv2.startWindowThread()
    # With that we can select a Region of Interest. Press Enter or Space to confirm the selection.
    # Performance on Mac is pretty bad for drawing/interacting, but works fine after that.
    ROI = cv2.selectROI("Select Bounding Box",cv2.cvtColor(musicVisualize, cv2.COLOR_BGR2RGB),showCrosshair=True,printNotice=True)
    searchBoxX=ROI[0]
    searchBoxY=ROI[1]
    searchBoxWidth=ROI[2]
    searchBoxHeight=ROI[3]
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    if searchBoxHeight<10 or searchBoxWidth<10:
        raise SystemExit("Invalid Input")

colorBox = (255, 255, 0)
colorTemplate = (0, 255, 0)

We build a list of templates used for matching the various symbols.

Pizzicati contains only one symbol (a cross), but Punkte has variations in: 1 Size, 2 Empty/Full, and 3 Normal, Single Strike, Double Strike.

In [389]:
templateSet=int(input("Enter the type of templates you want to use(1 Original Files, 2 Generated templates)"))

In [ ]:
# Build Templates
match ruleSet:
    #Punkte
    case 1:
        match templateSet:
            case 1:
                #Punkte
                partDot = musicTemplate[2360:2360+82, 1214:1214+80]
                cv2.rectangle(musicVisualizeTemplate, (1214, 2360), (1214+80, 2360+82), colorTemplate, 1)
                bigWhiteDot = musicTemplate[675:675+38, 1352:1352+34]
                cv2.rectangle(musicVisualizeTemplate, (1352, 675), (1352+34, 675+38), colorTemplate, 1)
                mediumWhiteDot = musicTemplate[684:684+26, 1440:1440+24]
                cv2.rectangle(musicVisualizeTemplate, (1440, 684), (1440+24, 684+26), colorTemplate, 1)
                smallWhiteDot = musicTemplate[782:782+21, 1438:1438+16]
                cv2.rectangle(musicVisualizeTemplate, (1438, 782), (1438+16, 782+21), colorTemplate, 1)
                bigBlackDot = musicTemplate[752:752+38, 964:964+34]
                cv2.rectangle(musicVisualizeTemplate, (964, 752), (964+34,752+38), colorTemplate, 1)
                mediumBlackDot = musicTemplate[813:813+27, 960:960+26]
                cv2.rectangle(musicVisualizeTemplate, (960, 813), (960+26, 813+27), colorTemplate, 1)
                smallBlackDot = musicTemplate[824:824+22, 1008:1008+20]
                cv2.rectangle(musicVisualizeTemplate, (1008, 824), (1008+20, 824+22), colorTemplate, 1)
                mediumWhiteDoubleStrikeDot = musicTemplate[1361:1361+40, 1874:1874+43]
                cv2.rectangle(musicVisualizeTemplate, (1874, 1361), (1874+43, 1361+40), colorTemplate, 1)
                #mediumWhiteDoubleStrikeDot = music[1369:1369+26, 1884:1884+24]
                mediumWhiteSingleStrikeDot = musicTemplate[993:993+28, 348:348+34]
                cv2.rectangle(musicVisualizeTemplate, (348, 993), (348+34, 993+28), colorTemplate, 1)
                mediumBlackDoubleStrikeDot = musicTemplate[1162:1162+38, 381:381+45]
                cv2.rectangle(musicVisualizeTemplate, (381, 1162), (381+45, 1162+38), colorTemplate, 1)
                mediumBlackSingleStrikeDot = musicTemplate[1161:1161+30, 1806:1806+45]
                cv2.rectangle(musicVisualizeTemplate, (1806, 1161), (1806+45, 1161+30), colorTemplate, 1)

                plt.figure(0)
                plt.title("Part Dot")
                plt.imshow(partDot, cmap="gray")
                plt.figure(1)
                plt.title("Big White Dot")
                plt.imshow(bigWhiteDot, cmap="gray")
                plt.figure(2)
                plt.title("Medium White Dot")
                plt.imshow(mediumWhiteDot, cmap="gray")
                plt.figure(3)
                plt.title("Small White Dot")
                plt.imshow(smallWhiteDot, cmap="gray")
                plt.figure(4)
                plt.title("Big Black Dot")
                plt.imshow(bigBlackDot, cmap="gray")
                plt.figure(5)
                plt.title("Medium Black Dot")
                plt.imshow(mediumBlackDot, cmap="gray")
                plt.figure(6)
                plt.title("Small Black Dot")
                plt.imshow(smallBlackDot, cmap="gray")
                plt.figure(7)
                plt.title("Medium White Double Strike Dot")
                plt.imshow(mediumWhiteDoubleStrikeDot, cmap="gray")
                plt.figure(8)
                plt.title("Medium White Single Strike Dot")
                plt.imshow(mediumWhiteSingleStrikeDot, cmap="gray")
                plt.figure(9)
                plt.title("Medium Black Double Strike Dot")
                plt.imshow(mediumBlackDoubleStrikeDot, cmap="gray")
                plt.figure(10)
                plt.title("Medium Black Single Strike Dot")
                plt.imshow(mediumBlackSingleStrikeDot, cmap="gray")
            case 2:
                MDTemplateGenerated = ski.io.imread('./templates/MWDTemplateGenerated.jpg')
                mediumWhiteDot = MDTemplateGenerated
                MDTemplateGenerated = ski.io.imread('./templates/MBDTemplateGenerated.jpg')
                mediumBlackDot = MDTemplateGenerated
                MDTemplateGenerated = ski.io.imread('./templates/WDDTemplateGenerated.jpg')
                mediumWhiteDoubleStrikeDot = MDTemplateGenerated
                MDTemplateGenerated = ski.io.imread('./templates/WSDTemplateGenerated.jpg')
                mediumWhiteSingleStrikeDot = MDTemplateGenerated
                MDTemplateGenerated = ski.io.imread('./templates/BDDTemplateGenerated.jpg')
                mediumBlackDoubleStrikeDot = MDTemplateGenerated
                MDTemplateGenerated = ski.io.imread('./templates/BSDTemplateGenerated.jpg')
                mediumBlackSingleStrikeDot = MDTemplateGenerated
                plt.figure(0)
                plt.title("Medium White Dot")
                plt.imshow(mediumWhiteDot, cmap="gray")
                plt.figure(1)
                plt.title("Medium Black Dot")
                plt.imshow(mediumBlackDot, cmap="gray")
                plt.figure(2)
                plt.title("Medium White Double Strike Dot")
                plt.imshow(mediumWhiteDoubleStrikeDot, cmap="gray")
                plt.figure(3)
                plt.title("Medium White Single Strike Dot")
                plt.imshow(mediumWhiteSingleStrikeDot, cmap="gray")
                plt.figure(4)
                plt.title("Medium Black Double Strike Dot")
                plt.imshow(mediumBlackDoubleStrikeDot, cmap="gray")
                plt.figure(5)
                plt.title("Medium Black Single Strike Dot")
                plt.imshow(mediumBlackSingleStrikeDot, cmap="gray")
            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")
            
    case 2:
        # Pizzicati
        match templateSet:
            case 2:
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated1.jpg')
                mediumCross = MCTemplateGenerated
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated2.jpg')
                mediumCross2 = MCTemplateGenerated
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated3.jpg')
                mediumCross3 = MCTemplateGenerated
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated4.jpg')
                mediumCross4 = MCTemplateGenerated
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated5.jpg')
                mediumCross5 = MCTemplateGenerated
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated6.jpg')
                mediumCross6 = MCTemplateGenerated
                plt.figure(0)
                plt.title("Medium Cross")
                plt.imshow(mediumCross, cmap="gray")
                plt.figure(1)
                plt.title("Medium Cross")
                plt.imshow(mediumCross2, cmap="gray")
                plt.figure(2)
                plt.title("Medium Cross")
                plt.imshow(mediumCross3, cmap="gray")
                plt.figure(3)
                plt.title("Medium Cross")
                plt.imshow(mediumCross4, cmap="gray")
                plt.figure(4)
                plt.title("Medium Cross")
                plt.imshow(mediumCross5, cmap="gray")
                plt.figure(5)
                plt.title("Medium Cross")
                plt.imshow(mediumCross6, cmap="gray")
            case 1:
                partDot = musicTemplate[2348:2348+84, 1217:1217+80]
                cv2.rectangle(musicVisualizeTemplate, (1217, 2348), (1217+80, 2348+84), colorTemplate, 1)
                mediumCross = musicTemplate[1207:1207+24, 1265:1265+30]
                cv2.rectangle(musicVisualizeTemplate, (1265, 1207), (1265+30, 1207+24), colorTemplate, 1)
                plt.figure(0)
                plt.title("Part Dot")
                plt.imshow(partDot, cmap="gray")
                plt.figure(1)
                plt.title("Medium Cross")
                plt.imshow(mediumCross, cmap="gray")
            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")
        
    case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
        raise SystemExit("Invalid Input")

Here we display the templates, and the chosen target image with the corresponding bounding box.

In [ ]:
# Bounding Box
cv2.rectangle(musicVisualize, (searchBoxX, searchBoxY), (searchBoxX+searchBoxWidth, searchBoxY+searchBoxHeight), colorBox, 2)

plt.figure(11,figsize=(15, 15))
plt.title("Templates")
plt.imshow(musicVisualizeTemplate)

plt.figure(12,figsize=(15, 15))
plt.title("Bounding Box")
plt.imshow(musicVisualize)

In [392]:
# First format the templates into a list of tuples (label, templateImage)
match ruleSet:
    case 1:
        # Punkte
        match templateSet:
            case 1:
                listTemplate = [('MWD', mediumWhiteDot), ('MBD', mediumBlackDot), ('WDD',mediumWhiteDoubleStrikeDot), ('WSD',mediumWhiteSingleStrikeDot), ('BDD',mediumBlackDoubleStrikeDot), ('BSD',mediumBlackSingleStrikeDot)]
         
                #listTemplate = [('BWD', bigWhiteDot), ('SWD', smallWhiteDot), ('BBD', bigBlackDot), ('SBD', smallBlackDot), ('MWD', mediumWhiteDot), ('MBD', mediumBlackDot),
                #               ('WDD', mediumWhiteDoubleStrikeDot), ('WSD', mediumWhiteSingleStrikeDot), ('BDD', mediumBlackDoubleStrikeDot), ('BSD', mediumBlackSingleStrikeDot)]
    
            case 2:
                listTemplate = [('MWD', mediumWhiteDot), ('MBD', mediumBlackDot), ('WDD',mediumWhiteDoubleStrikeDot), ('WSD',mediumWhiteSingleStrikeDot), ('BDD',mediumBlackDoubleStrikeDot), ('BSD',mediumBlackSingleStrikeDot)]


            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")                            
    case 2:
        # Pizzicati
        match templateSet:
            case 1:
                listTemplate = [('MC', mediumCross),]
            case 2:
                # listTemplate = [('MC1', mediumCross),('MC2', mediumCross2),('MC3', mediumCross3),('MC4', mediumCross4),('MC5', mediumCross5),('MC6', mediumCross6),]
                listTemplate = [('MC2', mediumCross2),('MC6', mediumCross6),]

            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")

    case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
        raise SystemExit("Invalid Input")

We generate an augmented template list. This means that on top of the original templates we produce scaled versions between 50% and 150%, 90 degree rotations, and mirroring on X and Y axis (and both).

For every template in the original list we build 176 templates in total. This is needed as we can potentially present the score in any orientation, and when using a different image we can have a different scaling (e.g. the distance of the image from the lens).

In [393]:
image_maxwh = musicSearch.shape
listTemplateSnapshot = listTemplate.copy()
listTemplate.clear()
for element in listTemplateSnapshot:
    for angles in range(0, 360, 90):
        for scales in range(50, 160, 10):
            # Scale and Rotate
            rotated = rotate(element[1], angle=angles)
            scaled, percentage = scale_image(rotated, scales, image_maxwh)
            listTemplate.append(('S '+str(angles)+' '+str(scales)+' '+element[0], scaled))
            listTemplate.append(('U '+str(angles)+' '+str(scales)+' '+element[0], np.flipud(scaled)))
            listTemplate.append(('L '+str(angles)+' '+str(scales)+' '+element[0], np.fliplr(scaled)))
            listTemplate.append(('UL '+str(angles)+' '+str(scales)+' '+element[0], np.flipud(np.fliplr(scaled))))

In [ ]:
# Then call the function matchTemplates (here a single template) setting the search box to fall inside the square
match ruleSet:
    case 1:
        # Punkte
        # Using BW Image 0.4 threshold works better, but with the original image you get too many false positives. 0.85 is more likely
        #listHits = matchTemplates(listTemplate, music, score_threshold=0.45,method=cv2.TM_CCOEFF_NORMED, maxOverlap=0.1, searchBox=(searchBoxX,searchBoxY, searchBoxWidth, searchBoxHeight))
        listHits = matchTemplates(listTemplate, musicSearch, score_threshold=0.65,method=cv2.TM_CCOEFF_NORMED, maxOverlap=0, searchBox=(searchBoxX,searchBoxY, searchBoxWidth, searchBoxHeight))
    
    case 2:
        # Pizzicati
        listHits = matchTemplates(listTemplate, musicSearch, score_threshold=0.75,method=cv2.TM_CCOEFF_NORMED, maxOverlap=0.1, searchBox=(searchBoxX,searchBoxY, searchBoxWidth, searchBoxHeight))

    case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
        raise SystemExit("Invalid Input")

print("Found {} hits".format(len(listHits)))

In [ ]:
Overlay = drawBoxesOnRGB(musicVisualize, listHits, boxColor=[255,0,0],showLabel=True, labelColor=(255,0,0))
plt.figure(figsize=(15, 15))
plt.imshow(Overlay)

In [396]:
# Prepare the lists of symbols sorted for the 4 parts
listSymbolsLeft=sorted(listHits, key=lambda tup: (tup[1][0],tup[1][1]) )
listSymbolsRight=sorted(listHits, key=lambda tup: (tup[1][0],tup[1][1]),reverse=True )
listSymbolsTop=sorted(listHits, key=lambda tup: (tup[1][1],tup[1][0]) )
listSymbolsBottom=sorted(listHits, key=lambda tup: (tup[1][1],tup[1][0]),reverse=True )

In [397]:
# Duration of the piece in seconds (1 to have it normalized between 0 and 1)
duration=1
# 1 Pixel duration in Seconds
onePixelDurLR=duration/searchBoxWidth
onePixelDurTB=duration/searchBoxHeight

# 1 Pixel in "MIDI Pitches"
onePixelPitchLR=127/searchBoxHeight
onePixelPitchTB=127/searchBoxWidth
MIDIFull=[]

In [398]:
# Part 1 (Left to Right X: Time Y: Pitch)
# First we rescale both Axis
MIDILeft=[]
for x in range (len(listSymbolsLeft)):
    listCoords=list(listSymbolsLeft[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round((listSymbolsLeft[x][1][0]-searchBoxX)*onePixelDurLR,10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxHeight+searchBoxY-listSymbolsLeft[x][1][1])*onePixelPitchLR))
    MIDILeft.append((listSymbolsLeft[x][0],(listCoords),listSymbolsLeft[x][2]))
MIDIFull.append(MIDILeft)

In [399]:
# Part 2 (Right to Left X: Time Y: Pitch)
# First we rescale both Axis
MIDIRight=[]
for x in range (len(listSymbolsRight)):
    listCoords=list(listSymbolsRight[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round(duration-((listSymbolsRight[x][1][0]-searchBoxX)*onePixelDurLR),10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxHeight+searchBoxY-listSymbolsRight[x][1][1])*onePixelPitchLR))
    MIDIRight.append((listSymbolsRight[x][0],(listCoords),listSymbolsRight[x][2]))
MIDIFull.append(MIDIRight)

In [400]:
# Part 3 (Top to Bottom X: Pitch Y: Time)
# First we rescale both Axis
MIDITop=[]
for x in range (len(listSymbolsTop)):
    listCoords=list(listSymbolsTop[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round(((listSymbolsTop[x][1][1]-searchBoxY)*onePixelDurTB),10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxWidth+searchBoxX-listSymbolsTop[x][1][0])*onePixelPitchTB))
    MIDITop.append((listSymbolsTop[x][0],(listCoords),listSymbolsTop[x][2]))
MIDIFull.append(MIDITop)

In [401]:
# Part 4 (Bottom to Top X: Pitch Y: Time)
# First we rescale both Axis
MIDIBottom=[]
for x in range (len(listSymbolsBottom)):
    listCoords=list(listSymbolsBottom[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round(duration-((listSymbolsBottom[x][1][1]-searchBoxY)*onePixelDurTB),10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxWidth+searchBoxX-listSymbolsBottom[x][1][0])*onePixelPitchTB))
    MIDIBottom.append((listSymbolsBottom[x][0],(listCoords),listSymbolsBottom[x][2]))
MIDIFull.append(MIDIBottom)

In [402]:
# Instruments and canonical MIDI range and program number
instrumentRange={
"Violin": (55,103,41, "Strings"),
"Viola": (48,91,42, "Strings"),
"Cello": (36,76,43, "Strings"),
"Double Bass": (28,67,44, "Strings"),
"Bass Guitar": (28,67,34, "Strings"),
"Acoustic Guitar": (40,88,26, "Strings"),
"Tuba": (28,58,59, "Brass"),
"Bass Trombone": (34,67,58, "Brass"),
"French Horn": (34,77,61, "Brass"),
"Trombone": (40,72,58, "Brass"),
"Trumpet": (55,82,57, "Brass"),
"Piccolo": (74,102,73, "Woodwinds"),
"Flute": (60,96,74, "Woodwinds"),
"Oboe": (58,91,69, "Woodwinds"),
"Alto Flute": (55,91,74, "Other"),
"English Horn": (52,81,70, "Other"),
"Clarinet": (50,94,72, "Woodwinds"),
"Bass Clarinet": (38,77,72, "Woodwinds"),
"Bassoon": (34,75,71, "Woodwinds"),
"Contrabassoon": (22,53,71, "Woodwinds"),
"Soprano Recorder": (72,98,75, "Other"),
"Alto Recorder": (65,91,75, "Other"),
"Tenor Recorder": (60,86,75, "Other"),
"Bass Recorder": (53,79,75, "Other"),
"Baritone Sax": (36,69,68, "Other"),
"Tenor Sax": (44,76,67, "Other"),
"Alto Sax": (49,81,66, "Other"),
"Soprano Sax": (56,88,65, "Other"),
"Glockenspiel": (79,108,10, "Other"),
"Xylophone": (65,108,14, "Other"),
"Vibraphone": (53,89,12, "Other"),
"Marimba": (45,96,13, "Other"),
"Bass Marimba": (33,81,13, "Other"),
"Celeste": (60,108,9, "Other"),
"Tubular Bells": (60,77,15, "Other"),
"Timpani": (40,55,48, "Other"),
"Harpsichord": (29,89,7, "Keyboard"),
"Harp": (24,103,47, "Other"),
"Piano": (21,108,1, "Keyboard"),
}

# Dynamic and average MIDI Velocity
dynamicRange={
"ppp":(16),
"pp":(32),
"p":(48),
"mp":(64),
"mf":(80),
"f":(96),
"ff":(112),
"fff":(127),
}

We finally want to build a MIDI rendering of the score, and a "transcription" with MusicXML.

One notable situation we need to address is that when we generate a NoteOn we need to generate a corresponding NoteOff somewhere in the future.

MIDI Files encode the timing of events as a delta from the previous event.

The solution is to build a data structure that contains note off with the absolute timing. Then sort it based on the timing attribute, and finally convert to delta timing by subtracting from the timing of the current event the timing of the previous event.

In [403]:
match scoreNumber:
        case 1|2:
                MIDIFileName='Punkte'+str(scoreNumber)
                noteDuration=32

        case 3|4:
                MIDIFileName='Pizzicati'+str(scoreNumber)
                noteDuration=64

        case 5:
                MIDIFileName=input("Enter the name of the file to save")
                # Individual note duration
                noteDuration=int(input("Enter minimum note duration (e.g 64 for 1/64th)"))
        case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")  

# Tempo in BPM
songTempo=60
# Duration at x BPM in sec
songDuration=180
songMin=songDuration/60
# Ticks per beat (conventionally quarter note)
ticksPerBeat=480

noteDelta=int(ticksPerBeat/(noteDuration/4))
# Compute the number of microseconds per tick (used for setting BPM) as the number of microseconds in 1 minute divided by BPM
microsecondsPerTick=int(60000000/songTempo)
# Conversion from 0-1 Time unit to Ticks
conversionTicks=int(ticksPerBeat*songTempo*songMin)

# List of instruments and order of appearance of the "directions"
instrumentList=['Violin','Cello','Flute','Piano']
trackOrder=['Left','Right','Top','Bottom']

# Create the complete MIDI File
midFull = MidiFile(ticks_per_beat=ticksPerBeat, type=1,)
for x in range(len(MIDIFull)):
        # Generate a new track
        track = MidiTrack()
        # Add it to the full file AND to a file for individual tracks
        midFull.tracks.append(track)
        midLocal=MidiFile(ticks_per_beat=ticksPerBeat, type=1,)
        midLocal.tracks.append(track)
        # Set the BPM in MIDI Track
        track.append(MetaMessage('set_tempo', tempo=microsecondsPerTick))
        # Set the instrument according to GM 1 (46 is Pizzicato Strings)
        instrument=instrumentRange[instrumentList[x]]
        track.append(Message('program_change', program = instrument[2], channel = 0))
        pitchCorrection=(instrument[1]-instrument[0])/127
        linearizedEvents=[]
        for event in MIDIFull[x]:
                # Here we need to process individual symbols that can translate to one or more notes
                currentPitch=round((event[1][1]*pitchCorrection)+instrument[0])
                match ruleSet:
                        case 1:
                                # In Punkte symbols can generate a sequence of 1 to 3 notes for all instruments, and on top of that can generate chords for piano
                                # Punkte also alters the dynamics
                                #Parse event[0]
                                tokens=event[0].split(" ")
                                # Token[2] gives us the "size" of the symbol
                                size=int(tokens[2])
                                # Token[3] gives us the type of symbol: plain, single or double strike
                                symbol=tokens[3]

                                clusterNum=1
                                if instrument[3]=='Keyboard':
                                       # For keyboards size is mapped to note clusters
                                       clusterNum=int(((size-50.0)/100.0)*(4-1)+1)
                                       # Velocity is mapped to black/white dots
                                       match symbol:
                                                case 'MWD'|'WSD'|'WDD':
                                                        velocity=32
                                                case 'MBD'|'BSD'|'BDD':
                                                        velocity=100
                                                case _:
                                                        # Anything else.
                                                        # 
                                                        # If `case _:` is omitted, an error will be thrown
                                                        # if `something` doesn't match any of the patterns.
                                                        raise SystemExit("Invalid Input")
                                else:
                                # For everything but keyboards size is mapped to dynamics
                                        velocity=int(((size-50.0)/100.0)*(dynamicRange['ff']-dynamicRange['pp'])+dynamicRange['pp'])

                                match symbol:
                                        case 'MWD'|'MBD':
                                                repetitions=1
                                        case 'WSD'|'BSD':
                                                repetitions=2
                                        case 'WDD'|'BDD':
                                                repetitions=3
                                        case _:
                                                # Anything else.
                                                # 
                                                # If `case _:` is omitted, an error will be thrown
                                                # if `something` doesn't match any of the patterns.
                                                raise SystemExit("Invalid Input")
                                for t in range(repetitions):
                                        for w in range(clusterNum):
                                                linearizedEvents.append(('note_on', currentPitch+w, velocity, int((event[1][0]*conversionTicks)+(noteDelta*(t)))))
                                                linearizedEvents.append(('note_off', currentPitch+w, 127, int(event[1][0]*conversionTicks)+(noteDelta*(t+1))))
                        case 2:
                                linearizedEvents.append(('note_on', currentPitch, 100, int(event[1][0]*conversionTicks)))
                                linearizedEvents.append(('note_off', currentPitch, 127, int(event[1][0]*conversionTicks)+noteDelta))

                        case _:
                                # Anything else.
                                # 
                                # If `case _:` is omitted, an error will be thrown
                                # if `something` doesn't match any of the patterns.
                                raise SystemExit("Invalid Input")  

        # Sort in ascending order of delta
        linearSortedEvents=sorted(linearizedEvents, key=lambda tup: (tup[3],tup[0]))

        for y in range(len(linearSortedEvents)):
                # Correct for Delta Time
                if (y==0):
                        track.append(Message(linearSortedEvents[y][0], note=linearSortedEvents[y][1], velocity=linearSortedEvents[y][2], time=linearSortedEvents[y][3]))
                        continue
                track.append(Message(linearSortedEvents[y][0], note=linearSortedEvents[y][1], velocity=linearSortedEvents[y][2], time=linearSortedEvents[y][3]-linearSortedEvents[y-1][3]))

        midLocal.save('./output/'+MIDIFileName+'_'+trackOrder[x]+'.mid')
        parsed = music21.converter.parse('./output/'+MIDIFileName+'_'+trackOrder[x]+'.mid',forceSource=True,quarterLengthDivisors=[16])
        fileHandle=parsed.write('musicxml',fp='./output/'+MIDIFileName+'_'+trackOrder[x]+'.musicxml')


midFull.save('./output/'+MIDIFileName+'_Full.mid')
parsed = music21.converter.parse('./output/'+MIDIFileName+'_Full.mid',forceSource=True,quarterLengthDivisors=[16])
fileHandle=parsed.write('musicxml',fp='./output/'+MIDIFileName+'_Full.musicxml')